### CR_1.11 - Padding Oracle

In [ ]:
from pwn import remote
from binascii import hexlify, unhexlify

BLOCK_SIZE = 16

def is_padding_ok(byte_padding_to_check, r):
    hex_padding = hexlify(byte_padding_to_check).decode()
    r.sendline(hex_padding.encode())
    response = r.recvline()
    padding_ok = b"Wow you are so strong at decrypting!" in response
    r.recv()
    return padding_ok

def attack(ciphertext, r):
    guessed_cleartext = b''

    def split_blocks(data, block_size):
        return [data[i:i + block_size] for i in range(0, len(data), block_size)]

    blocks = split_blocks(ciphertext, BLOCK_SIZE)
    
    for block_index in range(len(blocks) - 1, 0, -1):
        previous_block = blocks[block_index - 1]
        current_block = blocks[block_index]
        spliced_ciphertext = previous_block + current_block

        decoded_bytes = b'?' * BLOCK_SIZE

        for byte_index in range(BLOCK_SIZE - 1, -1, -1):
            padding_length = BLOCK_SIZE - byte_index

            hacked_ciphertext_tail = b''
            for padder_index in range(1, padding_length):
                hacked_ciphertext_tail += bytes([padding_length ^ decoded_bytes[byte_index + padder_index]])
            
            for guess in range(256):
                attack_byte = bytes([guess ^ spliced_ciphertext[byte_index]])
                hacked_ciphertext = spliced_ciphertext[:byte_index] + attack_byte + hacked_ciphertext_tail + spliced_ciphertext[byte_index + 1 + padding_length - 1:]

                if is_padding_ok(hacked_ciphertext, r):
                    test_correctness = hacked_ciphertext[:byte_index - 1] + bytes([1 ^ hacked_ciphertext[byte_index]]) + hacked_ciphertext[byte_index:]
                    if not is_padding_ok(test_correctness, r):
                        continue
                    
                    decoded_bytes = decoded_bytes[:byte_index] + bytes([hacked_ciphertext[byte_index] ^ padding_length]) + decoded_bytes[byte_index + 1:]
                    guessed_cleartext = bytes([guess ^ padding_length]) + guessed_cleartext
                    print(guessed_cleartext)
                    break
    
    return guessed_cleartext[:-guessed_cleartext[-1]]

if __name__ == '__main__':
    r = remote("padding.challs.cyberchallenge.it", 9033)
    try:
        r.recvuntil(b"flag\n")
        secret_flag = r.recvline().decode('utf-8').strip()
        r.recv()
        print(attack(bytes.fromhex(secret_flag), r))
    finally:
        r.close()

# CCIT{7h3_m057_f4m0u5_4774ck_0n_AES-CBC}

[x] Opening connection to padding.challs.cyberchallenge.it on port 9033
[x] Opening connection to padding.challs.cyberchallenge.it on port 9033: Trying 5.75.221.48
[+] Opening connection to padding.challs.cyberchallenge.it on port 9033: Done
b'\t'
b'\t\t'
b'\t\t\t'
b'\t\t\t\t'
b'\t\t\t\t\t'
b'\t\t\t\t\t\t'
b'\t\t\t\t\t\t\t'
b'\t\t\t\t\t\t\t\t'
b'\t\t\t\t\t\t\t\t\t'
b'}\t\t\t\t\t\t\t\t\t'
b'C}\t\t\t\t\t\t\t\t\t'
b'BC}\t\t\t\t\t\t\t\t\t'
b'CBC}\t\t\t\t\t\t\t\t\t'
b'-CBC}\t\t\t\t\t\t\t\t\t'
b'S-CBC}\t\t\t\t\t\t\t\t\t'
b'ES-CBC}\t\t\t\t\t\t\t\t\t'
b'AES-CBC}\t\t\t\t\t\t\t\t\t'
b'_AES-CBC}\t\t\t\t\t\t\t\t\t'
b'n_AES-CBC}\t\t\t\t\t\t\t\t\t'
b'0n_AES-CBC}\t\t\t\t\t\t\t\t\t'
b'_0n_AES-CBC}\t\t\t\t\t\t\t\t\t'
b'k_0n_AES-CBC}\t\t\t\t\t\t\t\t\t'
b'ck_0n_AES-CBC}\t\t\t\t\t\t\t\t\t'
b'4ck_0n_AES-CBC}\t\t\t\t\t\t\t\t\t'
b'74ck_0n_AES-CBC}\t\t\t\t\t\t\t\t\t'
b'774ck_0n_AES-CBC}\t\t\t\t\t\t\t\t\t'
b'4774ck_0n_AES-CBC}\t\t\t\t\t\t\t\t\t'
b'_4774ck_0n_AES-CBC}\t\t\t\t\t\t\t\t\t'
b'5_4774ck_0n_AES-CBC}\t

EOFError: 